Havasu Geometry ETL Example

Havasu, Iceberg, and WherobotsDB are all related to the management and processing of large-scale data, particularly in a data lake architecture, but they each have different roles and focuses. Here's a breakdown of their differences:

1. Havasu
Focus: Spatial data in data lakes.
Purpose: Havasu is a table format built on top of Apache Iceberg, but it specifically extends Iceberg to support geospatial data (both raster and vector data). It allows for efficient storage, querying, and management of spatial data, enabling complex operations on geospatial datasets in a scalable way.
Key Features:
Support for spatial data types (e.g., geometries and rasters).
Built for high-performance geospatial queries in data lakes.
Allows for ACID transactions, schema evolution, and time travel for spatial data.
Works with cloud storage solutions (e.g., AWS S3).
Designed to be cost-effective and scalable in managing large geospatial datasets.
2. Iceberg
Focus: Data lake management and big data storage.
Purpose: Apache Iceberg is a table format designed for large-scale analytics in cloud-based data lakes. It provides a high-performance foundation for managing structured data in data lakes by supporting features like partitioning, versioning, and schema evolution. It ensures that users can work with big datasets in a scalable and efficient way.
Key Features:
ACID transactions and snapshot isolation.
Schema evolution and time travel for data in data lakes.
Optimized for big data and analytics.
Supports multi-engine queries (e.g., Spark, Trino).
Allows for fine-grained partitioning for more efficient querying.
3. WherobotsDB
Focus: Spatial data processing and querying in a specialized database system.
Purpose: WherobotsDB is a spatial database optimized for geospatial queries. It is built to provide fast and scalable processing for large spatial datasets and is tightly integrated with Wherobots' ecosystem, making it useful for managing both vector and raster geospatial data efficiently. WherobotsDB is a spatial database that supports both raster and vector spatial data processing.
Key Features:
High-performance processing of spatial data, both raster and vector.
Spatial indexing and advanced querying capabilities for geospatial data.
Directly integrated with Wherobots’ geospatial analytics platform.
Optimized for large-scale geospatial data lakes.

config = SedonaContext.builder().appName('havasu-iceberg-geometry-etl')\
    .config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")\
    .getOrCreate()
sedona = SedonaContext.create(config)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
from sedona.spark import *

In [4]:
config = SedonaContext.builder().appName('havasu-iceberg-geometry-etl')\
    .config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")\
    .getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Load Taxi Pickup Records In WherobotsDB

In [5]:
taxidf = sedona.read.format('csv').option("header","true").option("delimiter", ",").load("s3://wherobots-examples/data/nyc-taxi-data.csv")

In [6]:
# Get the number of rows in the DataFrame
row_count = taxidf.count()

# Print the result
print(f"The number of rows in the DataFrame is: {row_count}")

The number of rows in the DataFrame is: 99998


In [7]:
taxidf.printSchema()

root
 |-- vendor_name: string (nullable = true)
 |-- Trip_Pickup_DateTime: string (nullable = true)
 |-- Trip_Dropoff_DateTime: string (nullable = true)
 |-- Passenger_Count: string (nullable = true)
 |-- Trip_Distance: string (nullable = true)
 |-- Start_Lon: string (nullable = true)
 |-- Start_Lat: string (nullable = true)
 |-- Rate_Code: string (nullable = true)
 |-- store_and_forward: string (nullable = true)
 |-- End_Lon: string (nullable = true)
 |-- End_Lat: string (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- Tip_Amt: string (nullable = true)
 |-- Tolls_Amt: string (nullable = true)
 |-- Total_Amt: string (nullable = true)



In [9]:
taxidf.show(3)

+-----------+--------------------+---------------------+---------------+-------------+----------+---------+---------+-----------------+----------+---------+------------+--------+---------+-------+-------+---------+---------+
|vendor_name|Trip_Pickup_DateTime|Trip_Dropoff_DateTime|Passenger_Count|Trip_Distance| Start_Lon|Start_Lat|Rate_Code|store_and_forward|   End_Lon|  End_Lat|Payment_Type|Fare_Amt|surcharge|mta_tax|Tip_Amt|Tolls_Amt|Total_Amt|
+-----------+--------------------+---------------------+---------------+-------------+----------+---------+---------+-----------------+----------+---------+------------+--------+---------+-------+-------+---------+---------+
|        VTS|         1/4/09 2:52|          1/4/09 3:02|              1|         2.63|-73.991957|40.721567|     NULL|             NULL|-73.993803|40.695922|        CASH|     8.9|      0.5|   NULL|      0|        0|      9.4|
|        VTS|         1/4/09 3:31|          1/4/09 3:38|              3|         4.55|-73.982102| 40

In [10]:
type(taxidf)

pyspark.sql.dataframe.DataFrame

In [11]:
taxidf = taxidf.selectExpr('ST_Point(CAST(Start_Lon AS Decimal(24,20)), CAST(Start_Lat AS Decimal(24,20))) AS pickup', 'Trip_Pickup_DateTime', 'Payment_Type', 'Fare_Amt')
taxidf = taxidf.filter(col("pickup").isNotNull())
taxidf.show(5)

+--------------------+--------------------+------------+--------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+--------------------+--------------------+------------+--------+
|POINT (-73.991957...|         1/4/09 2:52|        CASH|     8.9|
|POINT (-73.982102...|         1/4/09 3:31|      Credit|    12.1|
|POINT (-74.002587...|        1/3/09 15:43|      Credit|    23.7|
|POINT (-73.974267...|        1/1/09 20:52|      CREDIT|    14.9|
|POINT (-74.00158 ...|       1/24/09 16:18|        CASH|     3.7|
+--------------------+--------------------+------------+--------+
only showing top 5 rows



In [12]:
taxidf.createOrReplaceTempView('taxiDf')

Create or Replace a Temporary View:

createOrReplaceTempView() creates a temporary view based on the taxidf (DataFrame) in memory, which is available only for the current Spark session.
If a view with the name taxiDf already exists, it will be replaced with a new view based on taxidf.
Transform DataFrame into an SQL View:

After running this code, taxidf becomes accessible through SQL, meaning you can run SQL queries on it as if it were a table in a database.
Why would you do this?
Running SQL Queries:

After creating the temporary SQL view, you can use SQL queries to work with the data in the DataFrame. For example, you could run a query to select specific columns or sort the data.
This is particularly useful if you're more familiar with SQL than with the DataFrame API in Spark.
Combine SQL with Spark API:

Using SQL, you can combine SQL queries with Spark code and perform more complex operations like aggregation, filtering, sorting, etc.
Increased Flexibility:

By creating an SQL view, you can leverage more advanced SQL operations, which might be easier to write and understand compared to DataFrame operations.

Manage taxi pickup data using Havasu
Havasu is a data lake for geospatial data. Users can manage their datasets as Havasu tables.

Save DataFrame to a Havasu Table

In [13]:
sedona.sql("CREATE NAMESPACE IF NOT EXISTS wherobots.test_db")
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.taxi")
taxidf.writeTo("wherobots.test_db.taxi").create()

Read taxi pickup records from Havasu Table

In [14]:
taxidf = sedona.table("wherobots.test_db.taxi")
taxidf.show(5)
print('total count: ' + str(taxidf.count()))

+--------------------+--------------------+------------+--------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+--------------------+--------------------+------------+--------+
|POINT (-73.991957...|         1/4/09 2:52|        CASH|     8.9|
|POINT (-73.982102...|         1/4/09 3:31|      Credit|    12.1|
|POINT (-74.002587...|        1/3/09 15:43|      Credit|    23.7|
|POINT (-73.974267...|        1/1/09 20:52|      CREDIT|    14.9|
|POINT (-74.00158 ...|       1/24/09 16:18|        CASH|     3.7|
+--------------------+--------------------+------------+--------+
only showing top 5 rows

total count: 99998


pickup column is a geometry column. integration with WherobotsDB functions. directly apply Sedona ST_ functions to pickup column.

In [15]:
taxidf.printSchema()

root
 |-- pickup: geometry (nullable = true)
 |-- Trip_Pickup_DateTime: string (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: string (nullable = true)



In [16]:
taxidf.withColumn("buf", expr("ST_Buffer(pickup, 1e-4)")).show(5)

+--------------------+--------------------+------------+--------+--------------------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|                 buf|
+--------------------+--------------------+------------+--------+--------------------+
|POINT (-73.991957...|         1/4/09 2:52|        CASH|     8.9|POLYGON ((-73.991...|
|POINT (-73.982102...|         1/4/09 3:31|      Credit|    12.1|POLYGON ((-73.982...|
|POINT (-74.002587...|        1/3/09 15:43|      Credit|    23.7|POLYGON ((-74.002...|
|POINT (-73.974267...|        1/1/09 20:52|      CREDIT|    14.9|POLYGON ((-73.974...|
|POINT (-74.00158 ...|       1/24/09 16:18|        CASH|     3.7|POLYGON ((-74.001...|
+--------------------+--------------------+------------+--------+--------------------+
only showing top 5 rows



ACID Properties of Havasu Table
Havasu supports all ACID properties on a on-disk table, we can append data or modify the table.

In [17]:
bufdf = taxidf.withColumn("pickup", expr("ST_Buffer(pickup, 1e-4)"))

In [18]:
type(bufdf)

pyspark.sql.dataframe.DataFrame

In [19]:
bufdf.show(3)

+--------------------+--------------------+------------+--------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+--------------------+--------------------+------------+--------+
|POLYGON ((-73.991...|         1/4/09 2:52|        CASH|     8.9|
|POLYGON ((-73.982...|         1/4/09 3:31|      Credit|    12.1|
|POLYGON ((-74.002...|        1/3/09 15:43|      Credit|    23.7|
+--------------------+--------------------+------------+--------+
only showing top 3 rows



In [20]:
bufdf.writeTo("wherobots.test_db.taxi").append()

In [21]:
countAppend = sedona.table("wherobots.test_db.taxi").count()
print('total count after append: ' + str(countAppend))

total count after append: 199996


SQL to manipulate data

In [22]:
sedona.sql("SELECT * FROM wherobots.test_db.taxi").show(5)

+--------------------+--------------------+------------+--------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+--------------------+--------------------+------------+--------+
|POLYGON ((-73.991...|         1/4/09 2:52|        CASH|     8.9|
|POLYGON ((-73.982...|         1/4/09 3:31|      Credit|    12.1|
|POLYGON ((-74.002...|        1/3/09 15:43|      Credit|    23.7|
|POLYGON ((-73.974...|        1/1/09 20:52|      CREDIT|    14.9|
|POLYGON ((-74.001...|       1/24/09 16:18|        CASH|     3.7|
+--------------------+--------------------+------------+--------+
only showing top 5 rows



In [23]:
sedona.sql("INSERT INTO wherobots.test_db.taxi VALUES (ST_Point(10, 20), '1/26/09 10:20', 'Cash', 3.14)")
sedona.sql("INSERT INTO wherobots.test_db.taxi VALUES (ST_Point(10, 20), '1/26/09 10:20', 'Online', 31.4)")
sedona.sql("SELECT * FROM wherobots.test_db.taxi WHERE ST_Intersects(pickup, ST_Point(10, 20))").show()

+-------------+--------------------+------------+--------+
|       pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+-------------+--------------------+------------+--------+
|POINT (10 20)|       1/26/09 10:20|        Cash|    3.14|
|POINT (10 20)|       1/26/09 10:20|      Online|    31.4|
+-------------+--------------------+------------+--------+



In [24]:
sedona.sql("UPDATE wherobots.test_db.taxi SET Fare_Amt = 314 WHERE ST_Intersects(pickup, ST_Point(10, 20)) AND Payment_Type = 'Online'")
sedona.sql("SELECT * FROM wherobots.test_db.taxi WHERE ST_Intersects(pickup, ST_Point(10, 20))").show()

+-------------+--------------------+------------+--------+
|       pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+-------------+--------------------+------------+--------+
|POINT (10 20)|       1/26/09 10:20|      Online|     314|
|POINT (10 20)|       1/26/09 10:20|        Cash|    3.14|
+-------------+--------------------+------------+--------+



In [25]:
sedona.sql("DELETE FROM wherobots.test_db.taxi WHERE Payment_Type = 'Online'")
sedona.sql("SELECT * FROM wherobots.test_db.taxi WHERE ST_Intersects(pickup, ST_Point(10, 20))").show()

+-------------+--------------------+------------+--------+
|       pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+-------------+--------------------+------------+--------+
|POINT (10 20)|       1/26/09 10:20|        Cash|    3.14|
+-------------+--------------------+------------+--------+



table history and read a particular version of the table.

In [26]:
sedona.sql("SELECT * FROM wherobots.test_db.taxi.history ORDER BY made_current_at").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2025-03-22 17:38:...| 663094927212458863|               NULL|               true|
|2025-03-22 17:39:...|6724382584584544900| 663094927212458863|               true|
|2025-03-22 17:39:...|6084232593252143760|6724382584584544900|               true|
|2025-03-22 17:39:...|2834332403468061182|6084232593252143760|               true|
|2025-03-22 17:39:...|5632168637178731632|2834332403468061182|               true|
|2025-03-22 17:39:...|2570219443317877256|5632168637178731632|               true|
+--------------------+-------------------+-------------------+-------------------+



In [27]:
snapshots = sedona.sql("SELECT * FROM wherobots.test_db.taxi.history ORDER BY made_current_at").collect()

In [28]:
print(snapshots)

[Row(made_current_at=datetime.datetime(2025, 3, 22, 17, 38, 46, 872000), snapshot_id=663094927212458863, parent_id=None, is_current_ancestor=True), Row(made_current_at=datetime.datetime(2025, 3, 22, 17, 39, 7, 608000), snapshot_id=6724382584584544900, parent_id=663094927212458863, is_current_ancestor=True), Row(made_current_at=datetime.datetime(2025, 3, 22, 17, 39, 14, 610000), snapshot_id=6084232593252143760, parent_id=6724382584584544900, is_current_ancestor=True), Row(made_current_at=datetime.datetime(2025, 3, 22, 17, 39, 15, 853000), snapshot_id=2834332403468061182, parent_id=6084232593252143760, is_current_ancestor=True), Row(made_current_at=datetime.datetime(2025, 3, 22, 17, 39, 23, 866000), snapshot_id=5632168637178731632, parent_id=2834332403468061182, is_current_ancestor=True), Row(made_current_at=datetime.datetime(2025, 3, 22, 17, 39, 28, 653000), snapshot_id=2570219443317877256, parent_id=5632168637178731632, is_current_ancestor=True)]


In [29]:
snapshot_1 = snapshots[1]['snapshot_id']

In [30]:
print(snapshot_1)

6724382584584544900


In [31]:
sedona.table("wherobots.test_db.taxi").count()

199997

In [32]:
sedona.read.option("snapshot-id", snapshot_1).table("wherobots.test_db.taxi").count()

199996

In [33]:
sedona.sql("SELECT * FROM wherobots.test_db.taxi VERSION AS OF {}".format(snapshot_1)).count()

199996

 roll back to version 1

In [34]:
sedona.sql("CALL wherobots.system.rollback_to_snapshot('wherobots.test_db.taxi', {})".format(snapshot_1))
sedona.sql("SELECT * FROM wherobots.test_db.taxi").count()

199996

Optimize table for faster range query

In [35]:
predicate = "ST_Intersects(ST_PolygonFromEnvelope(-73.970730, 40.767844, -73.965615, 40.769217), pickup)"
taxidf = sedona.table("wherobots.test_db.taxi")
taxidf.where(predicate).count()

345

In [36]:
taxidf.show(3)

+--------------------+--------------------+------------+--------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+--------------------+--------------------+------------+--------+
|POINT (-73.991957...|         1/4/09 2:52|        CASH|     8.9|
|POINT (-73.982102...|         1/4/09 3:31|      Credit|    12.1|
|POINT (-74.002587...|        1/3/09 15:43|      Credit|    23.7|
+--------------------+--------------------+------------+--------+
only showing top 3 rows



CREATE SPATIAL INDEX on the table to sort the records by spatial proximity. Havasu supports sorting the geometry values by their Hilbert index.

In [37]:
sedona.sql("CREATE SPATIAL INDEX FOR wherobots.test_db.taxi USING hilbert(pickup, 16) OPTIONS map('target-file-count', '30')").show()

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|                         4|                    58|             27927950|                      0|
+--------------------------+----------------------+---------------------+-----------------------+



In [38]:
taxidf.show(3)

+-----------+--------------------+------------+--------+
|     pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+-----------+--------------------+------------+--------+
|POINT (0 0)|       1/27/09 14:41|        CASH|     6.1|
|POINT (0 0)|        1/25/09 2:51|        CASH|    12.1|
|POINT (0 0)|        1/26/09 7:23|        CASH|     7.3|
+-----------+--------------------+------------+--------+
only showing top 3 rows



The table displayed after creating the spatial index contains metrics about the Hilbert Curve index creation process for the wherobots.test_db.taxi table. Here's what each column means:

rewritten_data_files_count (4)

The number of data files that were rewritten during index creation.
In this case, 4 files were modified (e.g., split into smaller pieces or reorganized for more efficient access).

added_data_files_count (59)

The number of new files that were created as a result of indexing.
This means that the data was transformed into a more optimized format, allowing for faster spatial queries.

rewritten_bytes_count (27927950)

The amount of data (in bytes) that was rewritten during indexing.
In this case, approximately 27.9 MB of data was processed, which may include sorting, organizing, or creating new spatial structures. failed_data_files_count (0)

The number of files that failed to process.
A value of 0 means that the indexing process was successful, with no errors.

Spatial operations will run faster once you create a spatial index because the index organizes the data in a way that allows for more efficient searching and filtering.

The spatial index was created for the pickup column, which stores coordinates in the format POINT(lng lat). This will make queries that require filtering or location lookups (e.g., "find all pickups within 1 km of a given point") faster.

The same query scanned less data, this is because the spatial filter pushdown works better on sorted tables.

In [39]:
query = """
SELECT * 
FROM wherobots.test_db.taxi 
WHERE ST_Contains(ST_PolygonFromEnvelope(-74.01, 40.70, -73.95, 40.75), pickup)
"""
result = sedona.sql(query)
result.show()

+--------------------+--------------------+------------+--------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+--------------------+--------------------+------------+--------+
|POINT (-73.984397...|       1/24/09 15:34|        CASH|     6.1|
|POINT (-73.980453...|       1/23/09 13:29|        CASH|     5.7|
|POINT (-73.982037...|       1/16/09 20:52|        Cash|     7.4|
|POINT (-73.983719...|        1/4/09 11:01|      CREDIT|     5.3|
|POINT (-73.98482 ...|         1/7/09 1:12|        CASH|     8.1|
|POINT (-73.982475...|        1/4/09 13:56|        CASH|    10.5|
|POINT (-73.982367...|        1/15/09 8:28|      Credit|     4.9|
|POINT (-73.980468...|        1/1/09 16:05|        CASH|     6.5|
|POINT (-73.981487...|       1/30/09 20:21|      Credit|     5.7|
|POINT (-73.983719...|        1/25/09 2:00|        CASH|     7.3|
|POINT (-73.983252...|       1/15/09 19:56|        CASH|     4.1|
|POINT (-73.983428...|        1/25/09 0:19|      Credit|    11.7|
|POINT (-7

In [43]:
SedonaKepler.create_map(result, "question")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'question': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20…

In [40]:
query = """
SELECT * 
FROM wherobots.test_db.taxi 
WHERE ST_Distance(pickup, ST_Point(-73.985, 40.75)) < 0.01
"""
result = sedona.sql(query)
result.show()

+--------------------+--------------------+------------+--------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|
+--------------------+--------------------+------------+--------+
|POINT (-73.979408...|         1/6/09 2:06|        Cash|     5.4|
|POINT (-73.978168...|       1/23/09 15:31|        CASH|     6.1|
|POINT (-73.97692 ...|       1/24/09 14:40|        CASH|    12.1|
|POINT (-73.976892...|       1/12/09 16:24|        CASH|     4.9|
|POINT (-73.976493...|        1/18/09 7:52|        Cash|     6.1|
|POINT (-73.976307...|       1/11/09 10:31|        CASH|     7.3|
|POINT (-73.976083...|       1/11/09 21:25|        CASH|     8.9|
|POINT (-73.977888...|        1/9/09 16:15|        CASH|     7.7|
|POINT (-73.976565...|       1/21/09 16:39|        CASH|     6.5|
|POINT (-73.976202...|        1/6/09 16:11|        CASH|     5.7|
|POINT (-73.979692...|       1/16/09 13:21|        Cash|     8.5|
|POINT (-73.978862...|       1/28/09 19:41|        CASH|    16.1|
|POINT (-7

In [41]:
query = """
SELECT Payment_Type, COUNT(*) 
FROM wherobots.test_db.taxi 
WHERE ST_Within(pickup, ST_PolygonFromEnvelope(-74, 40.7, -73.95, 40.75)) 
GROUP BY Payment_Type
"""
result = sedona.sql(query)
result.show()

+------------+--------+
|Payment_Type|count(1)|
+------------+--------+
|   No Charge|     104|
|        CASH|   27332|
|      Credit|   11974|
|        Cash|   13945|
|     Dispute|      16|
|      CREDIT|     498|
+------------+--------+



When is the Hilbert Index Most Suitable for Spatial Analysis?
The Hilbert Index is particularly useful for efficiently sorting and filtering geospatial data. Here are the best use cases:

1️⃣ Nearest Neighbor Search
📌 Example:

Finding the closest restaurants to a user's location
Locating the nearest EV charging station
🔹 Why Hilbert Index?

The Hilbert Curve preserves spatial locality, meaning that nearby points in geographic space are also close in the index, significantly speeding up search operations.
2️⃣ Filtering and Aggregation in a Geographic Area (Range Queries)
📌 Example:

Retrieving all taxis within a 5 km radius of a given location
Analyzing the number of customers in a specific area
🔹 Why Hilbert Index?

Optimized spatial range searching – the index allows for efficient bounding box queries, reducing the number of records that need to be processed.
3️⃣ Density Analysis (Hotspot Detection, Heatmaps)
📌 Example:

Detecting high-crime areas
Analyzing vehicle traffic in cities
🔹 Why Hilbert Index?

Efficiently groups nearby points, making it easier to generate fast spatial statistics for dense data regions.
4️⃣ Spatio-Temporal Analysis (Tracking Changes Over Time)
📌 Example:

Monitoring land use changes over time
Tracking delivery routes at different times of the day
🔹 Why Hilbert Index?

It can be extended to index both spatial and temporal data, making it useful for trend analysis.
💡 Summary: When to Use the Hilbert Index?
✅ For nearest neighbor searches
✅ For optimizing queries within a specific geographic area
✅ For large-scale geospatial datasets (e.g., taxis, IoT, traffic data)
✅ When data is densely distributed and requires efficient indexing

Sorting by geohash value

In [42]:
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.taxi_sorted")
sortedTaxiDf = taxidf.withColumn("geohash", expr("ST_GeoHash(ST_Centroid(pickup), 20)"))\
    .sort(col("geohash"))

In [46]:
sortedTaxiDf.show(10)

+--------------------+--------------------+------------+--------+--------------------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|             geohash|
+--------------------+--------------------+------------+--------+--------------------+
|POINT (-0.03671 -...|        1/21/09 3:59|        CASH|     7.7|7zzzznvm28gp1qburdku|
|POLYGON ((-0.0366...|        1/21/09 3:59|        CASH|     7.7|7zzzznvm28gp1qburdku|
|POINT (-73.49519 ...|       1/26/09 22:02|        CASH|    15.7|dqgtrx43mvw34x9ny1dx|
|POLYGON ((-73.495...|       1/26/09 22:02|        CASH|    15.7|dqgtrx43mvw34x9ny1dx|
|POINT (-72.94221 ...|       1/22/09 22:31|        CASH|     4.9|dquju0mh5kf4773kb02u|
|POLYGON ((-72.942...|       1/22/09 22:31|        CASH|     4.9|dquju0mh5kf4773kb02u|
|POINT (-76.114692...|       1/29/09 23:49|        CASH|     8.5|dr1zefwyr5sn61w1n6zn|
|POLYGON ((-76.114...|       1/29/09 23:49|        CASH|     8.5|dr1zefwyr5sn61w1n6zn|
|POINT (-75.839898...|       1/27/09 18:04|

In [44]:
sortedTaxiDf.write.option("target-file-size-bytes", "1000000").format("havasu.iceberg").saveAsTable("wherobots.test_db.taxi_sorted")

In [45]:
sortedTaxiDf = sedona.table("wherobots.test_db.taxi_sorted")
sortedTaxiDf.where(predicate).count()

345

Geohash is a method of encoding geographic coordinates as a string of characters. Each geohash represents an area on the map, and its length determines the precision (the longer the geohash, the smaller the area).
ST_Centroid(pickup): Gets the center of a geometry (e.g. for a polygon).
ST_GeoHash(..., 20): Generates a 20-character geohash, which provides very high precision.
sort(col("geohash")): Sorts data by geohash, which improves the efficiency of spatial queries.

What is Geohash good for?

1️⃣ Effective filtering of geographic data
✅ If we want to find all the points in the area, just search for those that have a common geohash beginning (e.g. 7zzzznvm covers a larger area than 7zzzznvm28gp).

2️⃣ Better data organization (partitioning)
✅ Sorting by geohash helps group neighboring objects, which improves search speed and reduces the amount of data read.

3️⃣ Faster search in large data sets
✅ Geohash allows you to optimize indexing in databases such as Iceberg, Havasu or BigQuery.

4️⃣ Clustering and density analysis
✅ We can group data by geohash and analyze the density of objects in different places (e.g. where are the most taxi orders?).

In [48]:
sedona.sql("""
    SELECT * 
    FROM wherobots.test_db.taxi_sorted 
    WHERE geohash LIKE 'dr4p4j%'
""").show()

+--------------------+--------------------+------------+--------+--------------------+
|              pickup|Trip_Pickup_DateTime|Payment_Type|Fare_Amt|             geohash|
+--------------------+--------------------+------------+--------+--------------------+
|POINT (-75.839898...|       1/27/09 18:04|        CASH|     2.5|dr4p4jrndk934qjueyef|
|POLYGON ((-75.839...|       1/27/09 18:04|        CASH|     2.5|dr4p4jrndk934qjueyef|
+--------------------+--------------------+------------+--------+--------------------+



In [49]:
sedona.sql("""
    SELECT geohash, COUNT(*) AS num_trips, AVG(Fare_Amt) AS avg_fare
    FROM wherobots.test_db.taxi_sorted
    GROUP BY geohash
    ORDER BY num_trips DESC
""").show()

+--------------------+---------+------------------+
|             geohash|num_trips|          avg_fare|
+--------------------+---------+------------------+
|ebpbpbpbpbpbpbpbpbpb|     1427| 9.694393833216541|
|s0000000000000000000|     1427| 9.694393833216541|
|dr7grdqzppp0gd219ww8|      260|10.728461538461541|
|dr5rtpmfvxrz61cc4k74|       28| 7.421428571428573|
|dr5rsr32dz24ybq55uq6|       24| 9.066666666666668|
|dr5rwpg8ugdqmy816m49|       14| 6.614285714285715|
|dr5ru7fc444ek6rdswrg|       12|13.549999999999999|
|dr5rxth5nffb6vpcu73n|       12| 6.099999999999999|
|dr5rsrbnc9b6vpep69dc|        8|             21.75|
|dr5qugxsq3qzr3hrvus3|        8|               6.9|
|dr5rumb90u8ufh05vnmq|        8|               6.9|
|dr5rzjyt69x4jffgtmxv|        8| 7.199999999999999|
|dr5rs2gjwnknkx6y7tj2|        8|            10.775|
|dr5rveb37n4rhdhbs40d|        8|11.500000000000002|
|dr5rut20bbqq8sekgngw|        6| 8.233333333333333|
|dr5rtxzsp3xv3tux1dqn|        6|13.433333333333332|
|dr5rst1jdsg

Geohash and Hilbert Curve are two different approaches to indexing spatial data, each with its own strengths and weaknesses. Each algorithm has unique characteristics that make one more suitable for some use cases and the other for different ones. Here's a detailed comparison:

Geohash
Description:
Geohash is a method for encoding geographical space (latitude and longitude) into a string of alphanumeric characters. A geohash value is generated by encoding geographic coordinates into a single number that represents a region with a certain level of precision.

Use Cases:
Storing geographical points in a compact way.
Dividing space into cells of fixed size (squares) and representing these cells with geohash strings.
Proximity-based searches – Geohashes are often used for searching for points located near each other because they are naturally sorted spatially.
Advantages:
Simplicity – Easy to implement and fast in calculations.
Optimized for proximity searches – Geohash allows for quick searches of points in the same or neighboring cells.
Geospatial indexing – Useful for indexing large spatial datasets, such as points on a map.
Disadvantages:
Lack of precise boundaries – Geohash cells are square-shaped, which can lead to accuracy issues near the boundaries of cells.
Lack of locality within boundaries – Points close to the boundary of two geohashes can be far apart spatially, even though their geohash values are similar, which can be problematic for spatial analysis.
Hilbert Curve
Description:
The Hilbert Curve is a method of mapping a multidimensional space (e.g., 2D space) into a one-dimensional sequence of numbers while preserving the locality of points. It’s a space-filling curve that passes through every point in the space and maps a 2D space to a 1D space in a way that maintains the proximity of points.

Use Cases:
Transforming 2D space to 1D for efficient storage and indexing of spatial data.
Maintaining locality – It works very well in spatial analyses where proximity of points in space should be preserved in the encoding.
Spatial data querying – Used for efficiently performing spatial queries such as range queries or neighborhood queries.
Advantages:
Locality preservation – The Hilbert Curve maintains the proximity of points, which is essential for many spatial queries (e.g., finding points close to each other).
Better distribution of points – Compared to other techniques like geohash, the Hilbert Curve better maintains locality and avoids the "space splitting" problem found in geohash cells.
Scalability – Handles large spatial datasets well.
Disadvantages:
Computational complexity – While the Hilbert Curve provides better locality, its implementation is more complex than geohash.
Performance for small datasets – For small datasets, the difference in performance might be minimal, but for large datasets, Hilbert shows a significant advantage.

When to use which?
Geohash is suitable when:

You need a quick and simple way to index and search for geographical points in small or medium-sized datasets.
Data needs to be easy to store and transport (e.g., in mapping systems).
Your queries mostly concern geographic points and not complex spatial analyses.
Hilbert Curve is more suitable when:

You're working with large spatial datasets and need to preserve the proximity of points in space.
Your spatial queries are more complex (e.g., range queries, neighborhood searches, or spatial analyses).
You need higher precision in indexing and want to avoid space-splitting issues found with geohash.
In conclusion, geohash is faster and simpler, while the Hilbert Curve provides better results for large datasets and more complex spatial querie